In [1]:
import pandas as pd
from geopy.geocoders import Nominatim

In [2]:
# 지하철 정보 csv 파일 읽기
df_subway = pd.read_csv('./인프라 데이터/교통/서울시 역사마스터 정보.csv',encoding='cp949')
df_subway_loc = df_subway[["위도","경도"]]

In [3]:
#위도 경도로 주소 찾기
geo_local = Nominatim(user_agent= 'South Korea', timeout=None)
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y
    except:
        return [0, 0]
    
def geocoding_reverse(lat, lng):
    try:
        address = geo_local.reverse([lat, lng], exactly_one=True, language='ko')
        detail_address = address.address				# 상세주소
        zip_code = address.raw['address']['postcode']   # 우편번호
        x_y = detail_address.split(',')[:-2][::-1]
        x_y = ''.join(x_y)
        return x_y
    except:
        return [0, 0]

In [4]:
df_subway['주소'] = df_subway.apply(lambda row: geocoding_reverse(row['위도'], row['경도']),axis=1)
df_subway.head()


,역사_ID,역사명,호선,위도,경도,주소
0,9996,미사,5호선,37.560927,127.193877,하남시 미사1동 미사강변동로 70오페라약국
1,9995,강일,5호선,37.557490,127.175930,서울 강동구 상일2동 강일동 고덕로 산22-14강일
2,9010,동탄,수도권 광역급행철도,37.200340,127.095690,화성시 동탄신도시 동탄6동 오산동 경부고속도로경부동탄터널
3,9009,구성,수도권 광역급행철도,37.299130,127.103890,용인시 기흥구경부고속도로
4,9008,성남,수도권 광역급행철도,37.394670,127.120580,경기도 성남시 분당구 판교분당-수서 도시고속화도로


In [28]:
#주소에서 자치구 추출
df_subway['자치구'] = df_subway['주소'].str.extract(r'(서울 \w+구)')[0].str.split().str[1]
df_subway = df_subway.dropna()


In [42]:
#도로혼잡도 csv파일 읽기
df_traffic = pd.read_csv('./인프라 데이터/교통/도로혼잡도.csv', encoding='cp949')
df_traffic = df_traffic[df_traffic['signgu_cd'] < 20000]

In [12]:
#csv 파일 출력
df_traffic.to_csv('도로혼잡도.csv', encoding='utf-8', index=False)
